In [ ]:
from ultralytics import YOLO

In [ ]:
## Class labels in default model
model = YOLO("8-epoch-omelette.pt")
results = model.predict(source="./manual_test_data/omelette_1.jpeg")
model.names

In [ ]:
#detection of pre-specified classes
model = YOLO("yolov8n.pt")
results = model.predict(source="./manual_test_data/omelette_1.jpeg", classes=[45, 46, 47, 48, 49, 51, 52, 53, 54, 55], save=True)

In [ ]:
#measuring of time to predict
import time

model = YOLO("yolov8l.pt")
time_start = time.perf_counter()
results = model.predict(source="./manual_test_data/omelette_1.jpeg")
time_elapsed = (time.perf_counter() - time_start)
print("The task took", round(time_elapsed, 2), " seconds")

#results in seconds (local machine)
#nano: 0.46
#small: 0.87
#medium: 1.82
#large: 2.84
#xl: 4.13

In [ ]:
#evaluation of model
model.val()

In [ ]:
import numpy as np

#loading pre-trained model
#model = YOLO("path/to/model.pt")
model = YOLO("8-epoch-omelette.pt")

#make prediction
#param source = image to predict contents of
#param conf = confidence threshold
#param classes = classes included in search (9 food items and bowls)
#results = model.predict(source="./test_images/pizza.jpeg", conf=0.25, classes=[45, 46, 47, 48, 49, 51, 52, 53, 54, 55], save=True)
results = model.predict(source="./manual_test_data/omelette_3.jpeg", save=True)

#BRANCH 1: plate is detected - ration between food and plate
if any(results[0].boxes.cls == 45):
    #determine size of bowl (class 45)
    #box = results[0].boxes[results[0].boxes.cls == 45].xywh
    print(1)
    
#BRANCH 2: plate is not detected - size of food (assuming standard distance from food)
elif all(results[0].boxes.cls != 45):
    
    #calculate normalized width and height
    pos_tensor = results[0].boxes.xywhn
    
    #convert to numpy array
    pos_numpy = pos_tensor.detach().numpy()
    print(pos_numpy)
    

    #get min height / width for each object
    object_wh = np.delete(pos_numpy, [0,1], 1)
    object_height = (object_wh.min(axis=1)) / 2
    print(object_height)
    
    #calculate object area
    width_array = pos_numpy[:,2]
    height_array = pos_numpy[:,3]
    object_area = width_array * height_array 
    print (object_area)
    
    #calculate object volume
    factor_scaling = 0.75
    object_volume = object_area * object_height * factor_scaling

In [ ]:
    
#mask size detection
mask_object = results[0].masks # get Mask object
mask_shape = mask_object.shape
mask_segment = mask_object.segments # Bounding coordinates of masks -> List[segment] * N
mask_data = mask_object.data # raw masks tensor, (N, H, W)
# Iterate through the elements in tensor
for i in range(mask_data.shape[0]):
    zeros = (mask_data[i] == 0).sum()
    ones = (mask_data[i] == 1).sum()
    area_covered = ones / (zeros+ones) # Share of mask
    print ('Masks Raw. Mask No:', i, 'Class', results[0].boxes[i].cls, 'Area Covered:', area_covered, 'Shape:', mask_data[i].shape)
for i in range(len(mask_segment)):
    print ('Segment: Pixels in boundary:', len(mask_segment[i])) # No. of pixels in the boundary
print ('Pixels in Picture', int(mask_data.shape[1]) * int(mask_data.shape[2])) # No. of pixels in picture

In [ ]:
'''
Yolo training 
### Part 1 - yaml file with the following structure and information

path: ./food-data.yaml/
train: 'train/images'
val: 'valid/images'
 
# class names
names: 
  0: 'apple', 1: 'banana', 2: ...
  
### Part 2 - folder structure
├── data
## └── train
####└── images (folder including all training images)
####└── labels (folder including all training labels)
## └── test
####└── images (folder including all testing images)
####└── labels (folder including all testing labels)
## └── valid
####└── images (folder including all valid images)
####└── labels (folder including all valid labels)


### Part 3 - txt file with following data
Labels: Class label, X_Center, Y_Center, Width, Height
Example: [0 0.484167 0.222500 0.275000 0.351667]
'''